In [1]:
import sys
sys.path.append('./utils')

In [2]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import LSTM, Dropout, Merge, Embedding, Dense
from keras.models import Sequential
from keras import optimizers
from sklearn.model_selection import train_test_split
from metrics import evaluate_model
from embeddings import average_weights_embeddings, save_embeddings

Using TensorFlow backend.


In [3]:
DIM = 100
MAXLEN_SEQ = 550
DROPOUT = 0.2
BATCH_SIZE = 128
EPOCHS = 10
FILTERS = (8, 16, 32, 64)
KERNEL_SIZE = (4, 8, 10, 12)
BRANCHES = len(FILTERS)

In [4]:
df = pd.read_excel('training_set.xls')

In [5]:
df['target'] = df[['domain1_score', 'domain2_score']].apply(lambda x: np.nanmean(x), axis=1)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: Mean of empty slice
  """Entry point for launching an IPython kernel.


In [6]:
df = df[['essay', 'target']]

In [7]:
df.dropna(axis=0, inplace=True)

In [8]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['essay'])
sequences = tokenizer.texts_to_sequences(df['essay'])

In [9]:
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
print('Найдено %s уникальных токенов.' % len(word_index))

Найдено 41432 уникальных токенов.


In [10]:
X = pad_sequences(sequences, maxlen=MAXLEN_SEQ)

In [11]:
y = df['target'].astype('float32')

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
assert len(FILTERS) == len(KERNEL_SIZE)
submodels = []
for filters, kernel in zip(FILTERS, KERNEL_SIZE):
    submodel = Sequential()
    submodel.add(Embedding(vocab_size, 
                           DIM, 
                           name='emb-{}'.format(filters)))
    submodel.add(Dropout(DROPOUT))
    submodel.add(Conv1D(filters=filters,
                        kernel_size=kernel,
                        padding='same',
                        activation='relu'))
    submodel.add(MaxPooling1D(pool_size=2))
    submodels.append(submodel)
model = Sequential()
model.add(Merge(submodels, mode="concat"))
model.add(LSTM(128, return_sequences=True, dropout=DROPOUT))
model.add(LSTM(64, dropout=DROPOUT))
model.add(Dense(1, activation='relu'))
print('Compiling model')
model.compile(loss='mse',
              optimizer=optimizers.rmsprop(lr=0.002),
              metrics=['mae'])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  app.launch_new_instance()


Compiling model


In [14]:
model.fit([X_train] * BRANCHES,
          y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_data=([X_test] * BRANCHES, y_test))

Train on 10381 samples, validate on 2596 samples
Epoch 1/10
10381/10381 [==============================] - 57s - loss: 52.3089 - mean_absolute_error: 3.7541 - val_loss: 53.3716 - val_mean_absolute_error: 4.8859
Epoch 2/10
10381/10381 [==============================] - 55s - loss: 25.0509 - mean_absolute_error: 2.3797 - val_loss: 19.1682 - val_mean_absolute_error: 2.1142
Epoch 3/10
10381/10381 [==============================] - 55s - loss: 13.2665 - mean_absolute_error: 1.7417 - val_loss: 13.6613 - val_mean_absolute_error: 1.7711
Epoch 4/10
10381/10381 [==============================] - 54s - loss: 7.2875 - mean_absolute_error: 1.3650 - val_loss: 5.9018 - val_mean_absolute_error: 1.2794
Epoch 5/10
10381/10381 [==============================] - 54s - loss: 4.7221 - mean_absolute_error: 1.1626 - val_loss: 4.5198 - val_mean_absolute_error: 1.1273
Epoch 6/10
10381/10381 [==============================] - 55s - loss: 4.3719 - mean_absolute_error: 1.1112 - val_loss: 4.2529 - val_mean_absolute

In [15]:
evaluate_model(model, X_test, y_test, BRANCHES)

r2 score: 0.9417960480384787
explained variance score : 0.9439420700073242
kappa: 0.9687792624798369


In [16]:
embedding = average_weights_embeddings(FILTERS, model)

Усредняем веса 4 матриц весов эмбеддингов


In [17]:
save_embeddings(embedding, word_index, 'embeddings.txt')

Готово! Загружено 41432 векторов слов размерностью 100.
